In [1]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
sys.path.append(os.getcwd())
import tsa_utils
#import png

In [15]:
def build_model(self, data, labels, mode):
        print(data, labels, mode)
        if mode == tf.contrib.learn.ModeKeys.INFER:
            BATCH_SIZE=1
        else:
            BATCH_SIZE=30
        data = tf.reshape(data, [BATCH_SIZE, IMAGE_DEPTH, YSIZE, XSIZE, CHANNELS])
        conv1 = tf.layers.conv3d(inputs=data, filters=FILTER_COUNT, kernel_size=KERNEL_SIZE1, padding="same", strides=(DEPTHSTRIDE,XSTRIDE,YSTRIDE), name="conv1")
        pool1 = tf.layers.max_pooling3d(inputs=conv1, pool_size=POOLSIZE1, strides=POOL_STRIDES, name="pool1")
        conv2 = tf.layers.conv3d(inputs=pool1, filters=FILTER_COUNT, kernel_size=(2,3,3), padding="same", strides=(1, 1, 1), name="conv2", activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling3d(inputs=conv2, pool_size=POOLSIZE1, strides=POOL_STRIDES, name="pool2")
        conv3 = tf.layers.conv3d(inputs=pool2, filters=FILTER_COUNT, kernel_size=KERNEL_SIZE1, padding="same", strides=(DEPTHSTRIDE,XSTRIDE,YSTRIDE), name="conv3")
        pool3 = tf.layers.max_pooling3d(inputs=conv3, pool_size=POOLSIZE1, strides=POOL_STRIDES, name="pool1")
        conv4 = tf.layers.conv3d(inputs=pool3, filters=FILTER_COUNT, kernel_size=(2,3,3), padding="same", strides=(1, 1, 1), name="conv4", activation=tf.nn.relu)
        pool4 = tf.layers.max_pooling3d(inputs=conv4, pool_size=POOLSIZE2, strides=POOL_STRIDES, name="pool2")
        
        flat_pool = tf.reshape(pool2, [BATCH_SIZE, 133056])
        flat_pool=tf.identity(flat_pool, name="flat_pool")

        logits = tf.layers.dense(inputs=flat_pool, units=2, reuse=True)
        logits =tf.identity(logits, name="logits")
        logits = tf.reshape(logits, [BATCH_SIZE,2])

        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            flat_labels = tf.reshape(labels, [BATCH_SIZE, 2])
            test_labels=tf.identity(flat_labels, name="labels")
            class_weights=tf.reduce_sum(tf.multiply(flat_labels, tf.constant(WEIGHTS, dtype=tf.int64)), axis=1)
            loss = tf.losses.softmax_cross_entropy(onehot_labels=test_labels, logits=logits, weights=class_weights)
            train_op = tf.contrib.layers.optimize_loss(
                loss=loss,
                global_step=tf.contrib.framework.get_global_step(),
                learning_rate=LEARNING_RATE,
                optimizer="SGD")
        predictions = {
            "classes": tf.argmax(
              input=logits, axis=1, name="classes"),
          "probabilities": tf.nn.softmax(
              logits, name="softmax_tensor")}
        if mode == tf.contrib.learn.ModeKeys.INFER:
            return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions)
        return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [16]:
tsa_classifier = tf.contrib.learn.Estimator(model_fn=build_model, 
                                            model_dir="/home/cody/workspace/tsa_data/tsa3")
print(tsa_classifier.get_variable_names())
conv_filter = tsa_classifier.get_variable_value("dense/kernel")
print("CONV FILTER " +str(conv_filter))
print(tsa_classifier.get_variable_value("global_step"))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x9b0d650>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
['OptimizeLoss/learning_rate', 'conv1/bias', 'conv1/kernel', 'conv2/bias', 'conv2/kernel', 'conv3/bias', 'conv3/kernel', 'conv4/bias', 'conv4/kernel', 'dense/bias', 'dense/kernel', 'global_step']
CONV FILTER [[  2.31870264e-03   5.54288225e-03]
 [ -3.18676466e-05  -6.10276766e-04]
 [  2.55751400e-03  -2.75947241e-04]
 ..., 
 [ -4.47448750e-04  -3.50128999e-03]
 [ -5.16841328e-03  -4.70153149e-0

In [18]:

filters = tf.get_variable("dense/bias", shape=[1,4,4,4])

print(filters.shape)

x = filters[:,:,0,0]
#plt.imshow(x)
#plt.show()


# In[21]:

dir(tf.python_io)


ValueError: Variable dense/bias already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-13-14d45b9a1fe5>", line 2, in <module>
    filters = tf.get_variable("dense/bias", shape=[1])
  File "/home/cody/anaconda3/envs/p2.7/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/cody/anaconda3/envs/p2.7/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [29]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
import math 

sys.path.append(os.getcwd())
import tsa_utils
import deep_cnn
import mini_cnn

image_df = pd.read_csv( './images/stage1_labels.csv')
image_df['zone'] = image_df['Id'].str.split("_", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()
image_df = image_df.groupby(["id"]).sum()
image_df.loc[image_df['Probability'] == 0, 'probability'] = 0
image_df.loc[image_df['Probability'] >= 1, 'probability'] = 1

In [19]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

[u'dense/bias/Initializer/random_uniform/shape',
 u'dense/bias/Initializer/random_uniform/min',
 u'dense/bias/Initializer/random_uniform/max',
 u'dense/bias/Initializer/random_uniform/RandomUniform',
 u'dense/bias/Initializer/random_uniform/sub',
 u'dense/bias/Initializer/random_uniform/mul',
 u'dense/bias/Initializer/random_uniform',
 u'dense/bias',
 u'dense/bias/Assign',
 u'dense/bias/read',
 u'strided_slice/stack',
 u'strided_slice/stack_1',
 u'strided_slice/stack_2']